In [1]:
# Import all libraries

from __future__ import print_function
from __future__ import division
import os
import argparse
import torch
import torch.utils.data

import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
import json
import pytorch_crepe
import datetime
import numpy as np
import data_helpers
from CharCNN import CharCNN
np.random.seed(0123)


# set parameters:

subset = None

#Whether to save model parameters
save = False
model_name_path = 'params/crepe_model.json'
model_weights_path = 'params/crepe_model_weights.h5'


#Compile/fit params
batch_size = 50
nb_epoch = 10
maxlen = 256



print('Loading data...')
#Expect x to be a list of sentences. Y to be a one-hot encoding of the
#categories.
(xt, yt), (x_test, y_test) = data_helpers.load_ag_data()

print('Creating vocab...')
vocab, reverse_vocab, vocab_size, check = data_helpers.create_vocab_set()
test_data = data_helpers.encode_data(x_test, maxlen, vocab, vocab_size, check)



print('Build model...')

model = CharCNN()
model.cuda()
print(model)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01,momentum=0.9)



print('train model...')



Using TensorFlow backend.


Loading data...
Creating vocab...
Build model...
CharCNN (
  (features): Sequential (
    (0): Conv1d(69, 256, kernel_size=(7,), stride=(1,))
    (1): ReLU ()
    (2): MaxPool1d (size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(256, 256, kernel_size=(7,), stride=(1,))
    (4): ReLU ()
    (5): MaxPool1d (size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (6): Conv1d(256, 256, kernel_size=(3,), stride=(1,))
    (7): ReLU ()
    (8): Conv1d(256, 256, kernel_size=(3,), stride=(1,))
    (9): ReLU ()
    (10): Conv1d(256, 256, kernel_size=(3,), stride=(1,))
    (11): ReLU ()
    (12): Conv1d(256, 256, kernel_size=(3,), stride=(1,))
    (13): ReLU ()
    (14): MaxPool1d (size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential (
    (0): Dropout (p = 0.5)
    (1): Linear (1280 -> 1024)
    (2): ReLU (inplace)
    (3): Dropout (p = 0.5)
    (4): Linear (1024 -> 1024)
    (5): ReLU (inplace)
    (6): Linear (1024 -> 4)
  )
)
t

In [3]:
 for e in xrange(nb_epoch):
    xi, yi = data_helpers.shuffle_matrix(xt, yt)
    xi_test, yi_test = data_helpers.shuffle_matrix(x_test, y_test)
    if subset:
        batches = data_helpers.mini_batch_generator(xi[:subset], yi[:subset],
                                                    vocab, vocab_size, check,
                                                    maxlen,
                                                    batch_size=batch_size)
    else:
        batches = data_helpers.mini_batch_generator(xi, yi, vocab, vocab_size,
                                                    check, maxlen,
                                                    batch_size=batch_size)

    test_batches = data_helpers.mini_batch_generator(xi_test, yi_test, vocab,
                                                     vocab_size, check, maxlen,
                                                     batch_size=1)

    accuracy = 0.0
   
    step = 1
    start = datetime.datetime.now()
    print('Epoch: {}'.format(e))
    running_loss = 0.0
    i=0
    train_loss_avg=0.0
    train_correct=0.0
    for x_train, y_train in batches:
        i=i+1
        inputs=torch.from_numpy(np.swapaxes(x_train.astype(np.float64),1,2))
        labels=torch.from_numpy(np.argmax(y_train, axis=1).astype(np.float64))
        inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
        inputs=inputs.float()
        labels=labels.long()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        #train_loss += loss.data[0]
        train_loss_avg =loss.data.mean()
        _, predicted = torch.max(outputs.data, 1)
        train_correct = (predicted == labels.data).sum()
        accuracy +=  (train_correct*100/len(labels))
        accuracy_avg = accuracy / step
        if step % 100 == 0:
            print('  Step: {}'.format(step))
            print('\tLoss: {}. Accuracy: {}'.format(train_loss_avg,accuracy_avg))
        step += 1
    test_accuracy=0
    test_loss_avg=0
    test_correct=0
    step=1
    test_accuracy_avg=0
    for x_test_batch, y_test_batch in test_batches:
        inputs=torch.from_numpy(np.swapaxes(x_test_batch.astype(np.float64),1,2))
        labels=torch.from_numpy(np.argmax(y_test_batch, axis=1).astype(np.float64))
        inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
        inputs=inputs.float()
        labels=labels.long()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss_avg += loss.data[0]
        _, predicted = torch.max(outputs.data, 1)
        test_correct = (predicted == labels.data).sum()
        test_accuracy +=  (test_correct*100/len(labels))
        #test_accuracy_avg += (test_accuracy /step)
        step+=1
    print('Test Loss: {}. Test Accuracy: {}'.format(test_loss_avg/step,test_accuracy/step))
        


Epoch: 0
  Step: 100
	Loss: 1.38683974743. Accuracy: 24.74
  Step: 200
	Loss: 1.37997865677. Accuracy: 24.85
  Step: 300
	Loss: 1.38698852062. Accuracy: 24.9666666667
  Step: 400
	Loss: 1.38279271126. Accuracy: 24.905
  Step: 500
	Loss: 1.38365769386. Accuracy: 24.688
  Step: 600
	Loss: 1.3818192482. Accuracy: 24.9233333333
  Step: 700
	Loss: 1.39088129997. Accuracy: 24.96
  Step: 800
	Loss: 1.39429712296. Accuracy: 24.915
  Step: 900
	Loss: 1.38044857979. Accuracy: 24.9711111111
  Step: 1000
	Loss: 1.38008916378. Accuracy: 24.98
  Step: 1100
	Loss: 1.38725495338. Accuracy: 24.9054545455
  Step: 1200
	Loss: 1.39281094074. Accuracy: 24.9583333333
  Step: 1300
	Loss: 1.39067745209. Accuracy: 25.0015384615
  Step: 1400
	Loss: 1.38153719902. Accuracy: 25.0014285714
  Step: 1500
	Loss: 1.38250911236. Accuracy: 25.0333333333
  Step: 1600
	Loss: 1.39005494118. Accuracy: 25.05
  Step: 1700
	Loss: 1.38569116592. Accuracy: 25.0211764706
  Step: 1800
	Loss: 1.38031184673. Accuracy: 24.9866666667


  Step: 2300
	Loss: 0.391992300749. Accuracy: 77.727826087
  Step: 2400
	Loss: 0.351631015539. Accuracy: 77.87
Test Loss: 0.540943343715. Test Accuracy: 80.2920668333
Epoch: 6
  Step: 100
	Loss: 0.345025032759. Accuracy: 82.8
  Step: 200
	Loss: 0.715629935265. Accuracy: 83.14
  Step: 300
	Loss: 0.237282812595. Accuracy: 83.02
  Step: 400
	Loss: 0.516407251358. Accuracy: 82.915
  Step: 500
	Loss: 0.363313317299. Accuracy: 82.808
  Step: 600
	Loss: 0.513000488281. Accuracy: 83.0333333333
  Step: 700
	Loss: 0.438504070044. Accuracy: 83.1257142857
  Step: 800
	Loss: 0.57549059391. Accuracy: 83.31
  Step: 900
	Loss: 0.359960317612. Accuracy: 83.4711111111
  Step: 1000
	Loss: 0.566213905811. Accuracy: 83.524
  Step: 1100
	Loss: 0.474657595158. Accuracy: 83.6436363636
  Step: 1200
	Loss: 0.384576648474. Accuracy: 83.6566666667
  Step: 1300
	Loss: 0.330178290606. Accuracy: 83.7415384615
  Step: 1400
	Loss: 0.498018443584. Accuracy: 83.6842857143
  Step: 1500
	Loss: 0.358170002699. Accuracy: 83

In [2]:
k=[1,2,3,4,5]